In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
df = pd.read_csv('Dataset\Dataset.csv')

In [4]:
df.columns

Index(['customer_id', 'review_id', 'product_id', 'product_title',
       'product_category', 'rating', 'review_headline', 'review_body',
       'review_date'],
      dtype='object')

In [5]:
df['product_category'].unique

<bound method Series.unique of 0         Apparel
1         Apparel
2         Apparel
3         Apparel
4         Apparel
          ...    
6995    Gift Card
6996    Gift Card
6997    Gift Card
6998    Gift Card
6999    Gift Card
Name: product_category, Length: 7000, dtype: object>

In [6]:
df.head(10)

,customer_id,product_id,product_title,product_category,rating,review_date
0,50423057,B00JWXFDMG,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,31-08-2015
1,50423057,B00JWXFDMG,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,31-08-2015
2,33684654,B00VLMW18K,Scheppend Polymer Clay Cute Cartoon Fox Handma...,Jewelry,4,31-08-2015
3,33684654,B00VLMW18K,Scheppend Polymer Clay Cute Cartoon Fox Handma...,Jewelry,4,31-08-2015
4,11262325,B00W5T1H9W,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,31-08-2015
5,11262325,B00W5T1H9W,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,31-08-2015
6,41583822,B00UMGXJJQ,Claire's Accessories Girls Gold Chain Link Str...,Jewelry,5,31-08-2015
7,41583822,B00UMGXJJQ,Claire's Accessories Girls Gold Chain Link Str...,Jewelry,5,31-08-2015
8,27541121,B00M2L6KFY,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,31-08-2015
9,27541121,B00M2L6KFY,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,31-08-2015


In [6]:
# Sample user and item arrays (replace these with your own data)
users = df['customer_id']
items = df['product_id']
ratings = df['rating']

In [8]:
df['product_id'].head(15)

0     B00JWXFDMG
1     B00JWXFDMG
2     B00VLMW18K
3     B00VLMW18K
4     B00W5T1H9W
5     B00W5T1H9W
6     B00UMGXJJQ
7     B00UMGXJJQ
8     B00M2L6KFY
9     B00M2L6KFY
10    B00AJ2VU8A
11    B00AJ2VU8A
12    B0006SW2WU
13    B0006SW2WU
14    B00449ZF4O
Name: product_id, dtype: object

In [9]:
users.info()
items.info()
ratings.info()

<class 'pandas.core.series.Series'>
RangeIndex: 1194 entries, 0 to 1193
Series name: customer_id
Non-Null Count  Dtype
--------------  -----
1194 non-null   int64
dtypes: int64(1)
memory usage: 9.5 KB
<class 'pandas.core.series.Series'>
RangeIndex: 1194 entries, 0 to 1193
Series name: product_id
Non-Null Count  Dtype 
--------------  ----- 
1194 non-null   object
dtypes: object(1)
memory usage: 9.5+ KB
<class 'pandas.core.series.Series'>
RangeIndex: 1194 entries, 0 to 1193
Series name: rating
Non-Null Count  Dtype
--------------  -----
1194 non-null   int64
dtypes: int64(1)
memory usage: 9.5 KB


In [7]:
data = pd.DataFrame({'user': users, 'item': items, 'rating': ratings})

In [8]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
data['user'] = user_encoder.fit_transform(data['user'])
data['item'] = item_encoder.fit_transform(data['item'])

In [12]:
data.head()

,user,item,rating
0,91,78,5
1,91,78,5
2,70,133,4
3,70,133,4
4,36,134,5


In [9]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)


In [11]:
embedding_dim = 32

In [10]:
# Input layers for user and item
user_input = Input(shape=(1,))
item_input = Input(shape=(1,))

In [12]:
# Embedding layers
user_embedding = Embedding(input_dim=len(users), output_dim=embedding_dim)(user_input)
item_embedding = Embedding(input_dim=len(items), output_dim=embedding_dim)(item_input)


In [17]:
# Flatten embeddings
user_flatten = Flatten()(user_embedding)
item_flatten = Flatten()(item_embedding)

In [18]:
# Concatenate user and item embeddings
concatenated = Concatenate()([user_flatten, item_flatten])


In [19]:
# # Neural network layers
# hidden_layer = Dense(128, activation='relu')(concatenated)
# output_layer = Dense(1, activation='linear')(hidden_layer)
from keras.layers import Embedding, Input, Dense, Flatten, Concatenate, Dropout, BatchNormalization

#Additional hidden layers
hidden_layer_1 = Dense(128, activation='relu')(concatenated)
hidden_layer_1 = BatchNormalization()(hidden_layer_1)
hidden_layer_1 = Dropout(0.3)(hidden_layer_1)

# hidden_layer_2 = Dense(64, activation='relu')(hidden_layer_1)
# hidden_layer_2 = BatchNormalization()(hidden_layer_2)
# hidden_layer_2 = Dropout(0.3)(hidden_layer_2)

# hidden_layer_3 = Dense(32, activation='relu')(hidden_layer_2)
# hidden_layer_3 = BatchNormalization()(hidden_layer_3)
# hidden_layer_3 = Dropout(0.3)(hidden_layer_3)

# Output layer
output_layer = Dense(1, activation='linear')(hidden_layer_1)

In [20]:
# Create the model
model = Model(inputs=[user_input, item_input], outputs=output_layer)


In [21]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [22]:
model.layers

In [23]:
# Train the model
model.fit(
    [train_data['user'], train_data['item']],
    train_data['rating'],
    batch_size=64,
    epochs=100,
    validation_split=0.2
)

Epoch 1/100
12/12 [==============================] - 7s 181ms/step - loss: 17.2279 - val_loss: 15.9381
Epoch 2/100
12/12 [==============================] - 1s 99ms/step - loss: 15.0574 - val_loss: 14.6448
Epoch 3/100
12/12 [==============================] - 1s 76ms/step - loss: 13.4942 - val_loss: 13.2397
Epoch 4/100
12/12 [==============================] - 0s 8ms/step - loss: 11.6989 - val_loss: 11.7273
Epoch 5/100
12/12 [==============================] - 0s 8ms/step - loss: 10.1779 - val_loss: 10.1209
Epoch 6/100
12/12 [==============================] - 0s 21ms/step - loss: 8.4121 - val_loss: 8.4449
Epoch 7/100
12/12 [==============================] - 0s 9ms/step - loss: 6.5948 - val_loss: 6.7981
Epoch 8/100
12/12 [==============================] - 0s 20ms/step - loss: 5.1366 - val_loss: 5.3018
Epoch 9/100
12/12 [==============================] - 0s 21ms/step - loss: 3.5781 - val_loss: 4.0436
Epoch 10/100
12/12 [==============================] - 0s 20ms/step - loss: 2.4602 - val_loss

In [13]:
from tensorflow.keras.models import load_model
 
# load model
model = load_model('Models\WholeDatasetModel.h5')

In [14]:
# Evaluate the model
loss = model.evaluate([test_data['user'], test_data['item']], test_data['rating'])
print(f'Test Loss: {loss}')


44/44 [==============================] - 0s 1ms/step - loss: 0.0221
Test Loss: 0.022079268470406532


In [25]:
# test_data['user'] = user_encoder.transform(test_data['user'])
# test_data['item'] = item_encoder.transform(test_data['item'])

In [26]:
# Make predictions
predictions = model.predict([test_data['user'], test_data['item']])

# Convert predictions back to original ratings
predictions = np.clip(predictions, 1, 5)  # Clip predictions to rating range

# Print some predictions
for i in range(5):
    print(f"User: {test_data['user'].iloc[i]}, Item: {test_data['item'].iloc[i]}, Predicted Rating: {predictions[i][0]}")
#       print(f"User: {test_data['user']}, Product: {test_data['item']}, Predicted Rating:(predictions[i][0]).2f")
# Convert predictions back to original ratings
# predictions = np.clip(predictions, 1, 5)  # Clip predictions to rating range

# # Display recommendations for test data
# for i in range(len(test_data)):
#     user = test_data['user'].iloc[i]
#     item = test_data['item'].iloc[i]
#     predicted_rating = predictions[i][0]
#     product_name = "Appreal"# Replace with a function that gets the product name
#     print(f"User: {user}, Product: {product_name}, Predicted Rating: {predicted_rating:.2f}")

8/8 [==============================] - 0s 2ms/step
User: 78, Item: 66, Predicted Rating: 3.993959426879883
User: 70, Item: 133, Predicted Rating: 4.030628681182861
User: 60, Item: 123, Predicted Rating: 4.921962261199951
User: 89, Item: 119, Predicted Rating: 3.982926845550537
User: 17, Item: 96, Predicted Rating: 1.0


In [27]:
# just predicting rating -->

# Make predictions
predictions = model.predict([test_data['user'], test_data['item']])

# Convert predictions back to original ratings
predictions = np.clip(predictions, 1, 5)  # Clip predictions to rating range

# Map encoded user and item IDs back to their original values
decoded_users = user_encoder.inverse_transform(test_data['user'])
decoded_items = item_encoder.inverse_transform(test_data['item'])

# Print predictions with original user and item IDs
for i in range(25):
    user_id = decoded_users[i]
    item_id = decoded_items[i]
    predicted_rating = predictions[i][0]
    print(f"User ID: {user_id}, Item ID: {item_id}, Predicted Rating: {predicted_rating:.2f}")

8/8 [==============================] - 0s 2ms/step
User ID: 40689064, Item ID: B00EE3UOSW, Predicted Rating: 3.99
User ID: 33684654, Item ID: B00VLMW18K, Predicted Rating: 4.03
User ID: 28140828, Item ID: B00TYVQ3FM, Predicted Rating: 4.92
User ID: 46667015, Item ID: B00T24K90Q, Predicted Rating: 3.98
User ID: 1297040, Item ID: B00NHXOS6Y, Predicted Rating: 1.00
User ID: 28987335, Item ID: B00W9Q82T4, Predicted Rating: 3.94
User ID: 8280358, Item ID: B008OFYN9W, Predicted Rating: 2.21
User ID: 116982, Item ID: B005QL7IOY, Predicted Rating: 5.00
User ID: 44794333, Item ID: B00XVX9V8U, Predicted Rating: 4.87
User ID: 17350921, Item ID: B00EDS7OUY, Predicted Rating: 4.95
User ID: 31009358, Item ID: B00L0N0E8E, Predicted Rating: 4.95
User ID: 25810742, Item ID: B013E0AOH6, Predicted Rating: 5.00
User ID: 51793466, Item ID: B00WD82F1O, Predicted Rating: 4.88
User ID: 19993978, Item ID: B003O69NZ0, Predicted Rating: 1.39
User ID: 5377623, Item ID: B00CNU1C2A, Predicted Rating: 3.96
User ID: 

In [15]:

predictions = model.predict([test_data['user'], test_data['item']])
predictions = np.clip(predictions, 1, 5)  # Clip predictions to rating range
decoded_users = user_encoder.inverse_transform(test_data['user'])
decoded_items = item_encoder.inverse_transform(test_data['item'])
user_recommendations = {}

for i in range(len(test_data)):
    user_id = decoded_users[i]
    item_id = decoded_items[i]
    predicted_rating = predictions[i][0]
    
    if user_id not in user_recommendations:
        user_recommendations[user_id] = []
    
    user_recommendations[user_id].append((item_id, predicted_rating))

for user_id, recommendations in user_recommendations.items():
    recommendations.sort(key=lambda x: x[1], reverse=True)

top_n = 5  # Number of top recommendations to show
for user_id, recommendations in user_recommendations.items():
    print(f"User ID: {user_id}")
    num_recommendations = min(len(recommendations), top_n)  # Ensure not to go out of index
    for i in range(num_recommendations):
        item_id, predicted_rating = recommendations[i]
        print(f"  Recommendation {i + 1}: Item ID: {item_id}, Predicted Rating: {predicted_rating:.2f}")


44/44 [==============================] - 0s 948us/step
User ID: 24371595
  Recommendation 1: Item ID: B004LLIL5A, Predicted Rating: 4.84
  Recommendation 2: Item ID: B004LLIL5A, Predicted Rating: 4.84
  Recommendation 3: Item ID: B004LLIL5A, Predicted Rating: 4.84
User ID: 22138690
  Recommendation 1: Item ID: B00SENYY98, Predicted Rating: 1.08
User ID: 29173681
  Recommendation 1: Item ID: B00S4QN3YM, Predicted Rating: 3.91
User ID: 13319026
  Recommendation 1: Item ID: B014BYNZQQ, Predicted Rating: 4.91
  Recommendation 2: Item ID: B014BYNZQQ, Predicted Rating: 4.91
User ID: 37930676
  Recommendation 1: Item ID: B00JWABJRC, Predicted Rating: 4.91
User ID: 7899951
  Recommendation 1: Item ID: B005KPL7FM, Predicted Rating: 4.92
  Recommendation 2: Item ID: B005KPL7FM, Predicted Rating: 4.92
User ID: 30067162
  Recommendation 1: Item ID: B004LLIKVU, Predicted Rating: 4.86
  Recommendation 2: Item ID: B004LLIKVU, Predicted Rating: 4.86
  Recommendation 3: Item ID: B004LLIKVU, Predicted R

In [29]:
model.save('/kaggle/working/AugJewelry.h5')